<a href="https://colab.research.google.com/github/hansomatig/handwriting-synthesis/blob/TFConV1toV2/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git importieren


In [88]:
! git clone https://github.com/hansomatig/handwriting-synthesis.git

Cloning into 'handwriting-synthesis'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 184 (delta 70), reused 134 (delta 52), pack-reused 0
Receiving objects: 100% (184/184), 43.62 MiB | 24.22 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [ ]:
% cd handwriting-synthesis

In [ ]:
! git checkout TFConV1toV2

In [ ]:
! pip install svgwrite

In [ ]:
%load_ext autoreload
%autoreload 2
from demo import Hand

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

Model importieren



In [ ]:
from IPython.display import display, Image
import cairosvg
from keras.models import load_model
import PIL
import numpy as np

model = load_model('keras_model.h5')

lines = ["   Nick Pawlowski"]
biases = [2.5 for i in lines]
styles = [4 for i in lines]
hand = Hand()
predictions = {}

for i in range(20):
    hand.write(
        filename=f'img/img{i}.svg',
        lines=lines,
        styles=styles,
        biases=biases,
    )
    cairosvg.svg2png(url=f'img/img{i}.svg', write_to=f'img/png{i}.png')

    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    image = PIL.Image.open(f'img/png{i}.png')
    image = PIL.ImageOps.fit(image, (224, 224), PIL.Image.ANTIALIAS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array

    prediction = model.predict(data)
    predictions[i] = prediction[0][0]
    print(f'{i}/20')

predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)

for i in range(3):
    print(predictions[i][1]*100, '%')
    display(Image(filename=f'img/png{predictions[i][0]}.png'))